# To do:
* add in weighting option
* add function that bypasses the codebook to find variables based off search terms
* automatic variable cleaning?
* clean up existing functions

In [1]:
################### import packages ##########################################
import pandas as pd
import re
import numpy as np
import os
from hrs_functions import *
import copy 
import fnmatch
%load_ext line_profiler
pd.set_option('display.max_columns', None)

##############################################################################
####### choose variables to look for from codebooks ###########################
##############################################################################

# make sure these are all lowercase!!! #####################

# for HRS core files, 2002 - present
variables_to_look_for = { 'a500':'rmonth_survey'
                         , 'a501':'ryear_survey'
                         , 'p014':'rpr_lose_job'
                         , 'lb032_2':'rrisk_financial'
                         , 'n106':'rhosp_cost_oop'}

# for HRS core files, pre 2002
# 2000 has g's, 1998 has f's, 1996 has e's, 1994 has w's, 1992 has v's
variables_to_look_for_pre_2002 = {
      'g775': 'rmonth_survey'
    , 'f704': 'rmonth_survey'
    , 'e391': 'rmonth_survey'
    , 'w120': 'rmonth_survey'
    , 'v128': 'rmonth_survey'
    , 'g774': 'ryear_survey'
    , 'f703': 'ryear_survey'
    , 'e393': 'ryear_survey'
    , 'w122': 'ryear_survey'
    , 'v127': 'ryear_survey'
    , 'g4996': 'rpr_lose_job'
    , 'f4583': 'rpr_lose_job'
    , 'e3788': 'rpr_lose_job'
    , 'w5801': 'rpr_lose_job' 
    , 'v3205': 'rpr_lose_job'    
}

# for CAMS core files
CAMS_variables = { 'b1': 'rauto_pur_f' 
                , 'b2': 'rrefrig_pur_f' 
                , 'b3': 'rwasherdrier_pur_f'
                , 'b4': 'rdishwasher_pur_f'
                , 'b5': 'rtv_pur_f'}

# for HRS Rand file, choose patterns
#* is a wildcard for wave, so gets it for all waves
hrs_patterns = ['r*mstat', 'r*agey_b', 'h*itot']

# for HRS Rand file, choose patterns
cams_patterns = ['h*ctots', 'h*cdurs', 'h*cndur', 'h*ctotc']

# Assemble fat files

In [2]:
cwd = os.getcwd()
directory = cwd + '/data/HRS/'
years = [1992, 1994, 1996, 1998, 2000, 2002, 2004, 2006, 2008, 2010, 2012, 2014, 2016
        ]
output = read_all_years(years, directory, variables_to_look_for, variables_to_look_for_pre_2002)

reading files for survey year: 1992
looking for variables: ['g775', 'f704', 'e391', 'w120', 'v128', 'g774', 'f703', 'e393', 'w122', 'v127', 'g4996', 'f4583', 'e3788', 'w5801', 'v3205']
/Users/jcoven/Desktop/HRS_Assembly/python/data/HRS/h92sas/EMPLOYER
/Users/jcoven/Desktop/HRS_Assembly/python/data/HRS/h92sas/HEALTH
/Users/jcoven/Desktop/HRS_Assembly/python/data/HRS/h92sas/HHLIST
this is a pre-1996 household survey, so exclude
/Users/jcoven/Desktop/HRS_Assembly/python/data/HRS/h92sas/HOUSEHLD
this is a pre-1996 household survey, so exclude
/Users/jcoven/Desktop/HRS_Assembly/python/data/HRS/h92sas/KIDS
this is a pre-1996 household survey, so exclude
/Users/jcoven/Desktop/HRS_Assembly/python/data/HRS/h92sas/MODULEA
/Users/jcoven/Desktop/HRS_Assembly/python/data/HRS/h92sas/MODULEB
/Users/jcoven/Desktop/HRS_Assembly/python/data/HRS/h92sas/MODULEC
/Users/jcoven/Desktop/HRS_Assembly/python/data/HRS/h92sas/MODULED
/Users/jcoven/Desktop/HRS_Assembly/python/data/HRS/h92sas/MODULEE
/Users/jcoven/

/Users/jcoven/Desktop/HRS_Assembly/python/data/HRS/h04sas/H04P_R


reading files for survey year: 2006
looking for variables: ['ka500', 'ka501', 'kp014', 'klb032_2', 'kn106']
/Users/jcoven/Desktop/HRS_Assembly/python/data/HRS/h06sas/H06A_R
/Users/jcoven/Desktop/HRS_Assembly/python/data/HRS/h06sas/H06LB_R
/Users/jcoven/Desktop/HRS_Assembly/python/data/HRS/h06sas/H06L_R
/Users/jcoven/Desktop/HRS_Assembly/python/data/HRS/h06sas/H06N_R
/Users/jcoven/Desktop/HRS_Assembly/python/data/HRS/h06sas/H06P_R


reading files for survey year: 2008
looking for variables: ['la500', 'la501', 'lp014', 'llb032_2', 'ln106']
/Users/jcoven/Desktop/HRS_Assembly/python/data/HRS/h08sas/H08A_R
/Users/jcoven/Desktop/HRS_Assembly/python/data/HRS/h08sas/H08LB_R
/Users/jcoven/Desktop/HRS_Assembly/python/data/HRS/h08sas/H08L_R
/Users/jcoven/Desktop/HRS_Assembly/python/data/HRS/h08sas/H08N_R
/Users/jcoven/Desktop/HRS_Assembly/python/data/HRS/h08sas/H08P_R


reading files for survey year: 2010
looking for variables: ['

In [3]:
output['hhidpn'] = output['HHID'] + output['PN']
output.groupby('wave').count()

,HHID,PN,rpr_lose_job,ryear_survey,rmonth_survey,rhosp_cost_oop,rrisk_financial,hhidpn
wave,,,,,,,,
1.0,12877,12877,12652,12652,12652,0,0,12877
2.0,15928,15928,14439,15900,15900,0,0,15928
3.0,10964,10964,4623,10964,10964,0,0,10964
4.0,21384,21384,6030,21384,21384,0,0,21384
5.0,19580,19580,5175,19580,19580,0,0,19580
6.0,18167,18167,4220,18167,18167,2017,0,18167
7.0,20129,20129,5629,20129,20129,2306,0,20129
8.0,18469,18469,4797,18469,18469,2227,0,18469
9.0,17217,17217,0,17217,17217,2153,0,17217


# Read and reshape HRS Rand data

In [ ]:
# takes a 10-15 minutes to read because cant read this in a smart way
HRS_Rand = pd.read_stata(cwd + '/data/randhrs1992_2016v1_STATA/randhrs1992_2016v1.dta')

In [ ]:
# need to make this long instead of wide
long_hrs_rand = reshape_rand(HRS_Rand, hrs_patterns)

# make hhidpn a string, with leading zero's
long_hrs_rand['hhidpn'] = long_hrs_rand['hhidpn'].apply(lambda x: str(x).zfill(9))
long_hrs_rand['wave'] = long_hrs_rand['wave'].astype(float)
long_hrs_rand.groupby('wave').count()

# Assemble CAMS files

In [4]:
cwd = os.getcwd()
directory = cwd + '/data/CAMS/'
years = [2001, 2003, 2005, 2007, 2009, 2011, 2013, 2015, 2017
        ]
cams_output = read_all_years(years, directory, CAMS_variables, 'CAMS')

reading files for survey year: 2001
looking for variables: ['b1_01', 'b2_01', 'b3_01', 'b4_01', 'b5_01']
/Users/jcoven/Desktop/HRS_Assembly/python/data/CAMS/cams01/CAMS01_R


reading files for survey year: 2003
looking for variables: ['b1_03', 'b2_03', 'b3_03', 'b4_03', 'b5_03']
/Users/jcoven/Desktop/HRS_Assembly/python/data/CAMS/cams2003/CAMS03_R


reading files for survey year: 2005
looking for variables: ['b1_05', 'b2_05', 'b3_05', 'b4_05', 'b5_05']
/Users/jcoven/Desktop/HRS_Assembly/python/data/CAMS/cams2005/cams05_r


reading files for survey year: 2007
looking for variables: ['b1_07', 'b2_07', 'b3_07', 'b4_07', 'b5_07']
/Users/jcoven/Desktop/HRS_Assembly/python/data/CAMS/cams2007/CAMS07_R


reading files for survey year: 2009
looking for variables: ['b1_09', 'b2_09', 'b3_09', 'b4_09', 'b5_09']
/Users/jcoven/Desktop/HRS_Assembly/python/data/CAMS/cams2009/sas/CAMS09_R


reading files for survey year: 2011
looking for variables: ['b1_11', 'b2_11', 'b3_11', 'b4_11', 'b5_11']
/Users/j

In [10]:
cams_output['hhidpn'] = cams_output['HHID'] + cams_output['PN']
cams_output['wave'] = cams_output['wave'] - 0.5 #CAMS wave matching
cams_output.groupby('wave').count()

,HHID,PN,rauto_pur_f,rrefrig_pur_f,rwasherdrier_pur_f,rdishwasher_pur_f,rtv_pur_f,hhidpn
wave,,,,,,,,
5.0,3866,3866,3726,3732,3781,3778,3757,3866
6.0,3254,3254,3095,3104,3181,3186,3179,3254
7.0,5815,5815,3657,3676,3761,3781,3773,5815
8.0,5612,5612,3564,3626,3630,3632,3630,5612
9.0,5330,5330,3461,3533,3537,3542,3539,5330
10.0,6530,6530,4200,4295,4303,4303,4303,6530
11.0,6000,6000,3903,3989,4001,3997,3999,6000
12.0,5423,5423,3616,3688,3693,3692,3695,5423
13.0,5029,5029,3394,3447,3451,3454,3451,5029


# Read and reshape CAMS Rand data

In [6]:
# fast, small file
CAMS_Rand = pd.read_stata(cwd + '/data/randcams_2001_2017v1/randcams_2001_2017v1.dta')

In [8]:
# need to make this long instead of wide
long_cams_rand = reshape_rand(CAMS_Rand, cams_patterns)

# make hhidpn a string, with leading zero's
long_cams_rand['hhidpn'] = long_cams_rand['hhidpn'].apply(lambda x: str(x).zfill(9))
long_cams_rand['wave'] = long_cams_rand['wave'].astype(float)
long_cams_rand.groupby('wave').count()

,hhidpn,hctots,hcdurs,hcndur,hctotc
wave,,,,,
5.0,7624,3789,3789,3789,3789
6.0,7624,3224,3224,3224,3224
7.0,7624,3832,3832,3832,3832
8.0,7624,3704,3704,3704,3704
9.0,7624,3550,3550,3550,3550
10.0,7624,4319,4319,4319,4319
11.0,7624,4017,4017,4017,4017
12.0,7624,3689,3689,3689,3689
13.0,7624,3454,3454,3454,0


# Join data sources

In [ ]:
### join HRS core with HRS Rand

# hrs core files have 236962 rows
# hrs rand file has 546689 rows
# this is because the rand file has everyone for every year even if they didnt participate that year
# inner join has 236595 so something is up here
    # my guess is that because rand variables are made up of many variables
    # some people who participated didnt include all vars
    # and therefore dont have these rand variables so inner join is smaller
HRS_Core_and_Rand = output.merge(long_hrs_rand, on = ['hhidpn', 'wave'], how = 'outer')
HRS_Core_and_Rand.groupby('wave').count()

In [ ]:
### join HRS core and HRS Rand with CAMS Rand

# outer join because cams has fewer waves
HRS_Core_and_Rand_and_RANDCAMS = HRS_Core_and_Rand.merge(long_cams_rand, on = ['hhidpn', 'wave'], how = 'outer')
HRS_Core_and_Rand_and_RANDCAMS.groupby('wave').count()

In [ ]:
## join HRS core and HRS Rand and CAMS Rand with CAMS core
All_data = HRS_Core_and_Rand_and_RANDCAMS.merge(cams_output, on = ['hhidpn', 'wave'], how = 'outer')
All_data.groupby('wave').count()